# Circuit Construction 

The `Circuit` class forms the unit of computation that we send off to a quantum co-processor. Let's make the circuit.

In [1]:
from pytket import Circuit

trivial_circuit = Circuit()  # no qubits or bits
quantum_circ = Circuit(4)    # 4 qubits, no bits
mixed_circ = Circuit(2, 2)   # 2 qubits, 2 bits

named_circ = Circuit(2,2,"my_circuit")  # named circuit

## Basic Gates

Basic quantum gates represent some unitary operation applied to some qubits. Adding them to a `Circuit` just requires specifying which qubits you  want to apply them to. For controlled-gates, the convention is to give the control qubits first, followed by the target qubits.

In [2]:
from pytket import Circuit

circ = Circuit(4)  # qubits are numbered 0-3
circ.X(0)          # apply an X gate to qubit 0
circ.CX(1,3)       # and apply a CX gate with control qubit 1 and target qubit 3
circ.Z(3)          # apply a Z gate to qubit 3

circ.get_commands()  # get the list of gates in the circuit

[X q[0];, CX q[1], q[3];, Z q[3];]

For parameterised gates, such as rotations, the parameters is always given first. Because of the prevalence of rotations with anhles given by fractions of $\pi$ in practical quantum computing, the unit for all angular parameters is the half-turn. $1$ half turn is equal to $\pi$ radians)

In [3]:
from pytket import Circuit

circ = Circuit(2)

circ.Rx(0.5, 0)  # apply an Rx gate with angle 0.5, which is pi/2 to qubit 0
circ.CRz(0.3,1,0) # controlled Rz of angle 0.3pi, control qubit 1, target qubit 0

[Rx(0.5) q[0]; CRz(0.3) q[1], q[0]; ]

A large selection of common gates are available in this way. However, for less commonly used gates, a wider variety is available using the `OpType` enum, which can be added using the `Circuit.add_gate` method.

In [4]:
from pytket import Circuit, OpType

circ = Circuit(5)

circ.add_gate(OpType.CnX, [0,1,4,3])  # apply Controlled X gate with control qubit 0,1,4 and target qubit 3
circ.add_gate(OpType.XXPhase,0.7,[0,2])  # add e^{-i 0.7 pi / 2) XX} on qubits 0 and 2
circ.add_gate(OpType.PhasedX, [-0.1,0.5], [3])  # adds Rz(-0.5 pi) ; Rx(-0.1 pi) ; Rz(0.5 pi) on qubit 3

[CnX q[0], q[1], q[4], q[3]; XXPhase(0.7) q[0], q[2]; PhasedX(3.9, 0.5) q[3]; ]

In the above example, we asked for a `PhasedX` with angles `[-0.1, 0.5]`, but received `PhasedX(3.9, 0.5)`. `pytket` will freely map angles into the range $[0,r)$
for some range parameter
that depends on the `OpType`, preserving the unitary matrix (including global phase).

## Measurements

Measurements go a step further by interacting with both quantum and classical data. The convention used in `pytket` is that all measurements are non-destructive, single-qubit measurements in the $Z$ basis; other forms of measurements can be constructed by combining these with other operations.

Adding a measurement works just like adding any other gate, where the first argument is the qubit to be measured and the second specifies the classical bit store the result in.

In [5]:
from pytket import Circuit

circ = Circuit(4,2)
circ.Measure(0,0)  # measure qubit 0 into bit 0
circ.CX(1,2)
circ.CX(1,3)
circ.H(1)
circ.Measure(1,1)  # measurement of IXXX measure qubit 1 into bit 1

[Measure q[0] --> c[0]; CX q[1], q[2]; CX q[1], q[3]; H q[1]; Measure q[1] --> c[1]; ]

because the classical bits are treated as statically assigned locations, writing to the same bit multiple times will overwrite the previous value.

In [6]:
from pytket import Circuit

circ = Circuit(2,1)
circ.Measure(0,0) # the firrst measurement
circ.CX(0,1)
circ.Measure(1,0) # overwrites the first result with new measurement

[Measure q[0] --> c[0]; CX q[0], q[1]; Measure q[1] --> c[0]; ]

Depending on where we plan on running our circuits, the backend or simulator might have different requirements on the structure of measurements in the circuits. For example, statevector simulators will only work deterministically for pure-quantum circuits, so will fail if any measures are present at all. More crucially, near-term quantum hardware almost always requires all measurements to occur in a single parallel layer at the end of the circuit (i.e. we cannot measure a qubit in the middle of the circuit).

In [7]:
from pytket import Circuit

circ0 = Circuit(2,2) # all measurement at end
circ0.H(1)
circ0.Measure(0,0)
circ0.Measure(1,1)


circ1 = Circuit(2,2) # all measurement at end
circ1.Measure(0,0)
circ1.H(1)
circ1.Measure(1,1)


circ2 = Circuit(2,2) # all measurement at end
circ2.Measure(0,0)
circ2.CX(0,1)
circ2.Measure(1,1)

circ3 = Circuit(2,1) # all measurement at end
circ3.Measure(0,0)
circ3.Measure(1,0)


[Measure q[0] --> c[0]; Measure q[1] --> c[0]; ]

The simplest way to guarantee this is to finish the circuit by measuring all qubits. There is a short-hand function `Circuit.measure_all()` to make this easier.

In [8]:
from pytket import Circuit

# measure qubit 0 in Z basis and 1 in X basis
circ = Circuit(2, 2)
circ.H(1)
circ.measure_all()

# measure_all() adds bits if they are not already defined, so equivalently
circ = Circuit(2)
circ.H(1)
circ.measure_all()

[Measure q[0] --> c[0]; H q[1]; Measure q[1] --> c[1]; ]

On devices where mid-circuit measurements are available, they may be highly noisy and not apply just a basic projector on the quantum state. We can view these as “effectively destructive” measurements, where the qubit still exists but is in a noisy state. In this case, it is recommended to actively reset a qubit after measurement if it is intended to be reused.

In [9]:
from pytket import Circuit, OpType

circ = Circuit(2, 2)
circ.Measure(0, 0)
# Actively reset state to |0>
circ.add_gate(OpType.Reset, [0])
# Conditionally flip state to |1> to reflect measurement result
circ.X(0, condition_bits=[0], condition_value=1)
# Use the qubit as if the measurement was non-destructive
circ.CX(0, 1)

[Measure q[0] --> c[0]; Reset q[0]; IF ([c[0]] == 1) THEN X q[0]; CX q[0], q[1]; ]

## Barriers
The concept of barriers comes from low-level classical programming. They exist as instructions but perform no active operation. Instead, their function is twofold:

1. At compile time , prevent the compiler from reordering operations around the barrier
2. At runtime. ensure that all operations before the barrier must have finished before any operations after the barrier starts.

